In [1]:
import pandas as pd
import os
import math

input_path = '/home/dataconv/deallab/lasse/CCE_Data/raw_data/finecite/db_out_07_15'
ouput_path = '/home/dataconv/deallab/lasse/raw_data/finecite/'

doc_df =  pd.read_csv(os.path.join(input_path, 'documents_rows.csv'))
cited_docs_df =  pd.read_csv(os.path.join(input_path, 'cited_documents_rows.csv'))
par_df = pd.read_csv(os.path.join(input_path, 'paragraphs_rows.csv'))
ref_df = pd.read_csv(os.path.join(input_path, 'references_rows.csv'))
ann_df =  pd.read_csv(os.path.join(input_path, 'annotations_rows.csv'))

In [2]:
rows = [df.columns for df in [doc_df, cited_docs_df, par_df, ref_df, ann_df]]
rows

# INCLUDE CITATION LOCATION (REF_LOC) check
# INCLUDE -> always fill context1 first
# RENAME citation_location

[Index(['id', 'created_at', 'title', 'pub_year', 'abstract', 'doi', 'authors'], dtype='object'),
 Index(['ref_id', 'doc_id'], dtype='object'),
 Index(['created_at', 'doc_id', 'text', 'section_title', 'id'], dtype='object'),
 Index(['id', 'created_at', 'par_id', 'ref_loc', 'annotation_count',
        'iaa_total1', 'type', 'iaa_perc', 'iaa_inf', 'iaa_back', 'iaa_total2'],
       dtype='object'),
 Index(['id', 'created_at', 'user_id', 'ref_id', 'status', 'task_status',
        'comment', 'updated_at', 'context_location', 'guideline_version'],
       dtype='object')]

In [3]:
df = pd.DataFrame(columns=['id', 'citing_title', 'citing_pub_year', 'citing_authors', 'cited_title', 'cited_pub_year', 'cited_authors', 'citation_type', 'paragraph', 'target_reference_location', 'context_location1', 'context_location2', 'iaa_macro', 'iaa_total', 'iaa_inf', 'iaa_perc', 'iaa_back', 'guideline_version'])
annotated_ref = ann_df['ref_id'].to_list()

for idx, ref in ref_df.iterrows():
    id = ref['id']
    citation_type = ref['type']
    target_reference_location = ref['ref_loc']
    iaa_macro = ref['iaa_total1']
    iaa_total = ref['iaa_total2']
    iaa_inf = ref['iaa_inf']
    iaa_perc = ref['iaa_perc']
    iaa_back = ref['iaa_back']
    
    #annotation
    if id not in annotated_ref: continue
    anns = ann_df[ann_df['ref_id']==id].values
    #print(anns[0][4],anns[1][4])
    context_location1 = None
    context_location2 = None
    for ann in anns:
        if ann[8] != ann[8]: continue
        elif context_location1 is None: context_location1 = ann[8]
        elif context_location2 is None: context_location2 = ann[8]
        else: assert False, 'there are more then two annotated context for the reference'
    if context_location1 is None: continue  
    assert context_location1 is not None, 'There is no contet in this reference'
    guideline_version = anns[0][-1]
    
    #paragraph
    par = par_df[par_df['id']==ref['par_id']].values[0]
    paragraph = par[2]
    
    #citing_doc
    citing_doc = doc_df[doc_df['id']==par[1]].values[0]
    citing_title = citing_doc[2]
    citing_pub_year = citing_doc[3]
    citing_authors = citing_doc[6].replace(' ,', ';')
    #print(citing_title, citing_pub_year, citing_authors)
    
    #cited_doc
    cited_doc_ids = cited_docs_df[cited_docs_df['ref_id']==id]['doc_id'].to_list()
    cited_title = [doc_df[doc_df['id']==doc_id].values[0][2] for doc_id in cited_doc_ids]
    cited_pub_year = [doc_df[doc_df['id']==doc_id].values[0][3] for doc_id in cited_doc_ids]
    cited_authors = [doc_df[doc_df['id']==doc_id].values[0][6].replace(' ,', ';') for doc_id in cited_doc_ids]
    #print(cited_title, cited_pub_year, cited_authors)
    
    df.loc[len(df)] = [id, citing_title, citing_pub_year, citing_authors, cited_title, cited_pub_year, cited_authors, citation_type, paragraph, target_reference_location, context_location1, context_location2, iaa_macro, iaa_total, iaa_inf, iaa_perc, iaa_back, guideline_version]

print(len(df))
df.head()
    

1055


,id,citing_title,citing_pub_year,citing_authors,cited_title,cited_pub_year,cited_authors,citation_type,paragraph,target_reference_location,context_location1,context_location2,iaa_macro,iaa_total,iaa_inf,iaa_perc,iaa_back,guideline_version
0,00bcb525-6b6f-48be-8bd6-60852038c995,"Peru is Multilingual, Its Machine Translation ...",2021,Arturo Oncevay,[Google's multilingual neural machine translat...,[2017],[Melvin Johnson;Mike Schuster;Quoc Le;Maxim Kr...,single,Neural;Machine;Translation;(NMT);has;opened;se...,34,"[3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,1,1,1,1,1...",None,NaN,NaN,NaN,NaN,NaN,2.1
1,010f860b-24d4-4e94-b0d2-593693d657e6,The DCU Machine Translation Systems for IWSLT ...,2011,Pratyush Banerjee;Hala Almaghout;Sudip Naskar;...,[Experiments on Domain Adaptation for English-...,[2009],[R Haque;S Naskar;J Van Genabith;A Way],single,"As;shown;in;Table;1,;the;size;of;the;'in-domai...",42,"[0,0,0,0,0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,2...",None,NaN,NaN,NaN,NaN,NaN,2.1
2,014223dd-b870-48b7-9281-c9fbacf18f6f,A Language Invariant Neural Method for TimeML ...,2019,Suhan Prabhu;Pranav Goel;Alok Debnath;Manish S...,[Query based event extraction along a timeline...,"[2004, 2014]","[Hai Leong Chieu;Yoong Keok Lee, Goran Glavaš]",group,Automatic;extraction;of;events;has;gained;siza...,25,"[2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,1...",None,NaN,NaN,NaN,NaN,NaN,2.2
3,0157b9a7-b148-4d34-8d56-a4d827539085,A Parameter-Based Message-Passing Parser for M...,1994,Bonnie Dorr;Jye-Hoon Lee;Sungki Suh,[unknown],[unknown],[unknown],single,The;subject;NP;'Bill';is;coindexed;with;the;tr...,20,"[3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,2,2,2,2,0,0,1...",None,NaN,NaN,NaN,NaN,NaN,2.0
4,01b0446b-330d-4218-b91f-5a0e9e702d81,USST's System for AutoSimTrans 2022,2022,Jiahui Zhu;Jun Yu,[unknown],[unknown],[unknown],group,"Self-training;<ref type=""group"">(He et al., 20...",1,"[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0...",None,NaN,NaN,NaN,NaN,NaN,2.0


In [4]:
df.to_csv(os.path.join(ouput_path, 'full_data.csv'), index=False)